# Implicit Feedback Recommender Systems (ALS & BPR)

End-to-end implementation of **model-based collaborative filtering** using **implicit feedback**. The notebook covers data ingestion, sparse interaction modeling, matrix factorization, and ranking-based evaluation aligned with real-world recommender system pipelines.

## Data Preparation

User–item interactions are transformed into an implicit signal and represented as sparse matrices to enable scalable factorization methods.

In [ ]:
import pandas as pd
import numpy as np
import implicit
from scipy import sparse
import matplotlib.pyplot as plt
import time

In [ ]:
train_path = 'ml-100k/u3.base'
test_path = 'ml-100k/u3.test'

ratings = pd.read_csv(train_path, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings['implicit'] = (ratings['rating'] >= 3).astype(int)

users = ratings['user_id'].unique()
items = ratings['item_id'].unique()

user_map = {u: i for i, u in enumerate(users)}
item_map = {i: j for j, i in enumerate(items)}

rows = ratings['user_id'].map(user_map)
cols = ratings['item_id'].map(item_map)

interaction_matrix = sparse.csr_matrix((ratings['implicit'], (rows, cols)),
                                       shape=(len(users), len(items)))

## Model Training

Two complementary approaches are trained:
- Alternating Least Squares (ALS)
- Bayesian Personalized Ranking (BPR)

In [ ]:
als_model = implicit.als.AlternatingLeastSquares(
    factors=50,
    regularization=0.01,
    iterations=30,
    random_state=42
)

als_model.fit(interaction_matrix)

In [ ]:
bpr_model = implicit.bpr.BayesianPersonalizedRanking(
    factors=50,
    learning_rate=0.01,
    regularization=0.01,
    iterations=30,
    random_state=42
)

bpr_model.fit(interaction_matrix)

## Evaluation

Models are evaluated using ranking-based metrics aligned with top-N recommendation quality.

In [ ]:
from implicit.evaluation import mean_average_precision_at_k, ndcg_at_k

test_ratings = pd.read_csv(test_path, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
test_ratings['implicit'] = (test_ratings['rating'] >= 3).astype(int)

test_rows = test_ratings['user_id'].map(user_map).dropna().astype(int)
test_cols = test_ratings['item_id'].map(item_map).dropna().astype(int)

test_matrix = sparse.csr_matrix(
    (test_ratings.loc[test_rows.index, 'implicit'], (test_rows, test_cols)),
    shape=interaction_matrix.shape
)

map_als = mean_average_precision_at_k(als_model, test_matrix, interaction_matrix, K=10)
ndcg_als = ndcg_at_k(als_model, test_matrix, interaction_matrix, K=10)

map_bpr = mean_average_precision_at_k(bpr_model, test_matrix, interaction_matrix, K=10)
ndcg_bpr = ndcg_at_k(bpr_model, test_matrix, interaction_matrix, K=10)

print('ALS -> MAP@10:', map_als, 'NDCG@10:', ndcg_als)
print('BPR -> MAP@10:', map_bpr, 'NDCG@10:', ndcg_bpr)

## Key Takeaways

- Implicit-feedback modeling enables scalable personalization without explicit ratings.
- ALS provides strong baseline performance with stable convergence.
- BPR directly optimizes ranking quality through pairwise learning.